In [1]:
import numpy as np
import tensorflow as tf
import re
from collections import Counter
import json
from pprint import pprint
from tensorflow.contrib import learn
# from tokenize import *

import re
import csv


In [2]:
def tokenize_(s):
    pattern = r'''\d+|[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]|[A-Z][A-Z]+|http[s]?://[\w\./]+|[\w]+@[\w]+\.[\w]+|[a-z][a-z]+|[A-Za-z]\.[\w][\w\.]+|[\w]+|[-'a-z]+|[\S]+'''
    
    l = re.findall(pattern, s)
    return l


In [3]:
sentence = []
score = []

train_x = []
train_y = []

val_x = []
val_y = []

test_x = []
test_y = []

In [4]:
# filename = 'data.csv'

# #importing dataset into lists
# with open(filename, 'r') as f:
#     next(f)
#     reader = csv.reader(f, delimiter='$')
    
#     for row in reader:
#         score.append(int(row[0]))
#         sentence.append(row[1])

In [5]:
filename = 'train.csv'

#importing dataset into lists
with open(filename, 'r') as f:
    next(f)
    reader = csv.reader(f, delimiter='$')
    
    for row in reader:
        train_y.append([int(row[0])])
        train_x.append(row[1])
        sentence.append(row[1])

In [6]:
filename = 'valid.csv'

#importing dataset into lists
with open(filename, 'r') as f:
    next(f)
    reader = csv.reader(f, delimiter='$')
    
    for row in reader:
        val_y.append([int(row[0])])
        val_x.append(row[1])
        sentence.append(row[1])

In [7]:
filename = 'test.csv'

#importing dataset into lists
with open(filename, 'r') as f:
    next(f)
    reader = csv.reader(f, delimiter='$')
    
    for row in reader:
        test_y.append([int(row[0])])
        test_x.append(row[1])
        sentence.append(row[1])

In [8]:
words = []
for each in sentence:
#         each = each.lower()
#         words_this_sentence = each.split(' ')
    words.extend(tokenize_(each.lower()))

from collections import Counter
import random
print(len(words))

    
vocab = set(words)
vocab_size = len(vocab)
words_count = Counter(words)
sorted_vocab = sorted(words_count, key = words_count.get, reverse = True)


938509


In [9]:
filename = 'glove.6B.300d.txt'
def loadGloVe(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    return vocab,embd
vocab,embd = loadGloVe(filename)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

Loaded GloVe!


In [10]:
vocab = set(vocab)

word_to_int = {word:i for i,word in enumerate(vocab,1)}


In [11]:
################## to make features of the dataset #####################
sentence2ints = []
for each in sentence:
    each = tokenize_(each)
    this_sentence_int = []
    for word in each:
        if word in vocab:
            this_sentence_int.append(word_to_int[word])
    sentence2ints.append(this_sentence_int)

max_seq_len = 20
features = np.zeros((len(sentence2ints), max_seq_len), dtype=int)
for i, row in enumerate(sentence2ints):
    features[i, :len(row)] = np.array(row[:max_seq_len] )
#######################################################################


In [12]:
# #Shuffling Dataset
# from random import shuffle
# data = list(zip(features, score))
# shuffle(data)
# features = [ item[0] for item in data]
# score = [ [item[1]] for item in data]


# #Splitting
# split_frac = 0.6
# split_idx = int(len(features)*split_frac)

# train_x, val_x = features[:split_idx], features[split_idx:]
# train_y, val_y = score[:split_idx], score[split_idx:]

# test_idx = int(len(val_x)*0.5)
# val_x, test_x = val_x[:test_idx], val_x[test_idx:]
# val_y, test_y = val_y[:test_idx], val_y[test_idx:]



In [13]:
############# NEW GLOVE ################

W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]),
                trainable=False, name="W")
embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])
embedding_init = W.assign(embedding_placeholder)

#######################################

In [14]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
n_epochs = 1


In [15]:
X = tf.placeholder(tf.int32, [None, None], name = 'inputs')
Y = tf.placeholder(tf.float32, [None, 1], name = 'labels')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [16]:
embed = tf.nn.embedding_lookup(W, X)

# print (embed)
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

cell = tf.contrib.rnn.MultiRNNCell([lstm]*lstm_layers)

#getting an initial state of zeros\n",
initial_state = cell.zero_state(batch_size, tf.float32)

outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state = initial_state)


In [22]:
embed

<tf.Tensor 'embedding_lookup:0' shape=(?, ?, 300) dtype=float32>

In [18]:
predictions = tf.contrib.layers.fully_connected(outputs[:, -1],1, activation_fn=tf.tanh)

loss = tf.reduce_mean(tf.square(Y - predictions))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

#Accuracy:
correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.float32), Y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(embedding_init, feed_dict={embedding_placeholder: embedding})

array([[ 0.04656   ,  0.21318001, -0.0074364 , ...,  0.0090611 ,
        -0.20988999,  0.053913  ],
       [-0.25538999, -0.25723001,  0.13169   , ..., -0.23289999,
        -0.12226   ,  0.35499001],
       [-0.12559   ,  0.01363   ,  0.10306   , ..., -0.34224001,
        -0.022394  ,  0.13684   ],
       ..., 
       [ 0.075713  , -0.040502  ,  0.18345   , ...,  0.21838   ,
         0.30967   ,  0.43761   ],
       [ 0.81450999, -0.36221001,  0.31186   , ...,  0.075486  ,
         0.28408   , -0.17558999],
       [ 0.42919099, -0.29689699,  0.15011001, ...,  0.28975001,
         0.32618001, -0.0590532 ]], dtype=float32)

In [21]:
# sess.run(tf.global_variables_initializer())
for e in range(n_epochs):
    state = sess.run(initial_state)
    iteration = 1

    for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
        feed = {X: x, Y: y, initial_state: state}

        embed_temp, state, loss_,  _ = sess.run([embed, final_state, loss, optimizer], feed_dict=feed)
        print(embed_temp.shape)
        if iteration%5==0:
            print("Epoch: {}/{}".format(e, n_epochs),
                  "Iteration: {}".format(iteration),
                  "Train loss: {:.3f}".format(loss_))

        if iteration%25==0:
            val_acc = []
            val_state = sess.run(cell.zero_state(batch_size, tf.float32))
            for x, y in get_batches(val_x, val_y, batch_size):
                feed = {X: x,
                        Y: y,
                        initial_state: val_state}
                batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                val_acc.append(batch_acc)
            print("Val acc: {:.5f}".format(np.mean(val_acc)))

            train_acc = []

            train_state = sess.run(cell.zero_state(batch_size, tf.float32))
            for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
                feed = {X: x,Y: y,initial_state: train_state}

                batch_acc, train_state = sess.run([accuracy, final_state], feed_dict=feed)
                train_acc.append(batch_acc)

            print("Train accuracy: {:.5f}".format(np.mean(train_acc)))

        iteration +=1

print('Training Completed')

test_acc = []

test_state = sess.run(cell.zero_state(batch_size, tf.float32))
for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
    feed = {X: x,Y: y,initial_state: test_state}

    batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
    test_acc.append(batch_acc)

print("Test accuracy: {:.5f}".format(np.mean(test_acc)))

ValueError: invalid literal for int() with base 10: 'Please provide your registered mobile number'